In [1]:
import os
import urllib
import zipfile
import numpy as np
import torch
import sklearn.metrics 
from torch_geometric.utils import dense_to_sparse
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from functools import partial, reduce
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch_geometric_temporal.signal import temporal_signal_split

from typing import Union, Tuple
import math
import datetime
from tqdm import tqdm
from torch_geometric_temporal.signal.static_graph_temporal_signal import StaticGraphTemporalSignal
from torch_geometric_temporal.signal.dynamic_graph_temporal_signal import DynamicGraphTemporalSignal
from torch_geometric_temporal.signal.dynamic_graph_static_signal import DynamicGraphStaticSignal

from torch_geometric_temporal.signal.static_graph_temporal_signal_batch import StaticGraphTemporalSignalBatch
from torch_geometric_temporal.signal.dynamic_graph_temporal_signal_batch import DynamicGraphTemporalSignalBatch
from torch_geometric_temporal.signal.dynamic_graph_static_signal_batch import DynamicGraphStaticSignalBatch

from torch_geometric_temporal.signal.static_hetero_graph_temporal_signal import StaticHeteroGraphTemporalSignal
from torch_geometric_temporal.signal.dynamic_hetero_graph_temporal_signal import DynamicHeteroGraphTemporalSignal
from torch_geometric_temporal.signal.dynamic_hetero_graph_static_signal import DynamicHeteroGraphStaticSignal

from torch_geometric_temporal.signal.static_hetero_graph_temporal_signal_batch import StaticHeteroGraphTemporalSignalBatch
from torch_geometric_temporal.signal.dynamic_hetero_graph_temporal_signal_batch import DynamicHeteroGraphTemporalSignalBatch
from torch_geometric_temporal.signal.dynamic_hetero_graph_static_signal_batch import DynamicHeteroGraphStaticSignalBatch

Discrete_Signal = Union[
    StaticGraphTemporalSignal,
    StaticGraphTemporalSignalBatch,
    DynamicGraphTemporalSignal,
    DynamicGraphTemporalSignalBatch,
    DynamicGraphStaticSignal,
    DynamicGraphStaticSignalBatch,
    StaticHeteroGraphTemporalSignal,
    StaticHeteroGraphTemporalSignalBatch,
    DynamicHeteroGraphTemporalSignal,
    DynamicHeteroGraphTemporalSignalBatch,
    DynamicHeteroGraphStaticSignal,
    DynamicHeteroGraphStaticSignalBatch,
]
import random

# GPU support
DEVICE = torch.device('cuda') # cuda



In [2]:
class COVID19Dataloader(object):


    def __init__(self, raw_data_dir=os.path.join(os.getcwd(), "data_covid_weekly")):
        super(COVID19Dataloader, self).__init__()
        self.raw_data_dir = raw_data_dir
        self._read_web_data()

    def _download_url(self, url, save_path):  # pragma: no cover
        with urllib.request.urlopen(url) as dl_file:
            with open(save_path, "wb") as out_file:
                out_file.write(dl_file.read())

    def _read_web_data(self):

        A = np.load(os.path.join(self.raw_data_dir, "adj_matrix_corr_adj_0.6.npy"))
        X = np.load(os.path.join(self.raw_data_dir, "node_features_weekly_date.npy")).transpose(
            (1, 2, 0)
        )
        X_data = X.astype(np.float64)
        X_dates = X.astype(np.float64)
        
        means = np.mean(X_data, axis=(0, 2))
        X_data = X_data - means.reshape(1, -1, 1)
        stds = np.std(X_data, axis=(0, 2))
        X_data = X_data / stds.reshape(1, -1, 1)
        
        self.A = torch.from_numpy(A)   
        self.X_data = torch.from_numpy(X_data)
        self.X_dates = torch.from_numpy(X_dates)
        self.means = means
        self.stds = stds

    def _get_edges_and_weights(self):
        edge_indices, values = dense_to_sparse(self.A)
        edge_indices = edge_indices.numpy()
        values = values.numpy()
        self.edges = edge_indices
        self.edge_weights = values
        

    def _generate_task(self, num_timesteps_in, num_timesteps_out):
        """Uses the node features of the graph and generates a feature/target
        relationship of the shape
        (num_nodes, num_node_features, num_timesteps_in) -> (num_nodes, num_timesteps_out)
        predicting the COVID-19 weekly deaths using num_timesteps_in to predict the
        deaths in the next num_timesteps_out

        Args:
            num_timesteps_in (int): number of timesteps the sequence model sees
            num_timesteps_out (int): number of timesteps the sequence model has to predict
        """
        indices = [
            (i, i + (num_timesteps_in + num_timesteps_out))
            for i in range(self.X_data.shape[2] - (num_timesteps_in + num_timesteps_out) + 1)
        ]
        

        # Generate observations
        features, target = [], []
        dates = []
        for i, j in indices:
            features.append((self.X_data[:, 1:, i : i + num_timesteps_in]).numpy())
            target.append((self.X_data[:, 1, i + num_timesteps_in : j]).numpy())
            dates.append((self.X_dates[:, 0, i + num_timesteps_in : j]).numpy())

        self.features = features
        self.targets = target
        self.dates = dates


    def get_dataset(
        self, num_timesteps_in, num_timesteps_out
    ) -> StaticGraphTemporalSignal:
        """Returns data iterator for covid19 dataset as an instance of the
        static graph temporal signal class.

        Return types:
            * **dataset** *(StaticGraphTemporalSignal)* - The COVID19
                forecasting dataset.
        """
        self._get_edges_and_weights()
        self._generate_task(num_timesteps_in, num_timesteps_out)
        dataset = StaticGraphTemporalSignal(
            self.edges, self.edge_weights, self.features, self.targets
        )

        return dataset,self.dates,self.means,self.stds

In [3]:
def data_loader(past_values,future_values):
    loader = COVID19Dataloader()
    dataset,dates,means,stds = loader.get_dataset(past_values,future_values)
    death_stds = stds[1]
    death_mean = means[1]
    
    return dataset,dates,death_stds,death_mean

In [4]:
# print("Number of samples / sequences: ",  len(tuple(dataset)))
# print(next(iter(dataset))) # Show first sample

In [5]:
def temporal_signal_split_def(
    data_iterator,dates,window, train_ratio,batch_size
) -> Tuple[Discrete_Signal, Discrete_Signal]:
    r"""Function to split a data iterator according to a fixed ratio.
    Arg types:
        * **data_iterator** *(Signal Iterator)* - Node features.
        * **train_ratio** *(float)* - Graph edge indices.
    Return types:
        * **(train_iterator, test_iterator)** *(tuple of Signal Iterators)* - Train and test data iterators.
    """
    #dataiterator = whole dataset
    length = data_iterator.snapshot_count
    train_snapshots = int(train_ratio * length)   
    train_dataset = data_iterator[window:train_snapshots+window]
    test_dataset = data_iterator[train_snapshots+window:train_snapshots+window+batch_size]
    test_dates = dates[train_snapshots+window:train_snapshots+window+batch_size]
    
    for snapshot in train_dataset:
        static_edge_index = snapshot.edge_index.to(DEVICE)
        edge_weights = snapshot.edge_attr.to(DEVICE)
        break;    
    
    train_input = np.array(train_dataset.features)
    train_target = np.array(train_dataset.targets)

    train_x_tensor = torch.from_numpy(train_input).type(torch.FloatTensor).to(DEVICE)  # (B, N, F, P)
    train_target_tensor = torch.from_numpy(train_target).type(torch.FloatTensor).to(DEVICE)  # (B, N, T)

    train_dataset_new = torch.utils.data.TensorDataset(train_x_tensor, train_target_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset_new, batch_size=batch_size, shuffle=False,drop_last=True)
    
    test_input = np.array(test_dataset.features)
    test_target = np.array(test_dataset.targets)

    test_x_tensor = torch.from_numpy(test_input).type(torch.FloatTensor).to(DEVICE)  # (B, N, F, P)
    test_target_tensor = torch.from_numpy(test_target).type(torch.FloatTensor).to(DEVICE)  # (B, N, T)

    test_dataset_new = torch.utils.data.TensorDataset(test_x_tensor, test_target_tensor)
    test_loader = torch.utils.data.DataLoader(test_dataset_new, batch_size=batch_size, shuffle=False,drop_last=True)
    
    return train_loader, test_loader,static_edge_index,edge_weights,test_dates

In [6]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods, past_values,batch_size,hidden_units):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        self.tgnn = A3TGCN2(in_channels=node_features,  out_channels=hidden_units, periods=past_values, batch_size=batch_size
                            ,add_self_loops=False)

        self.linear = torch.nn.Linear(hidden_units, periods)
       

    def forward(self, x, edge_index,edge_weight):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index,edge_weight) # x [b, 49, 2, 15]  returns h [b, 49, 14]
        h = F.relu(h) 
        h = self.linear(h)
        
        
        return h

In [7]:
def train(node_f,periods,batch_size,past_values,hidden_units,trainratio,test_ratio,epochs,dataset,dates):
    model = TemporalGNN(node_features=node_f, periods=periods, past_values = past_values, batch_size=batch_size,hidden_units=hidden_units).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = torch.nn.MSELoss()
    samples = len(tuple(dataset))
    pred_test = []

    for i in tqdm(range(math.ceil(test_ratio*samples)-(batch_size))):

        results = pd.DataFrame(columns = ['date'])
        window = i +1

        train_loader_d, test_loader_d, static_edge_index_d,edge_weights_d,test_dates = temporal_signal_split_def(dataset,dates, i, train_ratio=trainratio,
                                                                                                                 batch_size=batch_size)
        results['date'] = pd.to_datetime(test_dates[0][0], format='%Y%m%d')

        for epoch in range(epochs):
            model.train()    
            step = 0
            loss_list = []
            for encoder_inputs_train, labels_train in train_loader_d:

                y_hat_train = model(encoder_inputs_train, static_edge_index_d,edge_weights_d) # Get model predictions

                loss_train = loss_fn(y_hat_train, labels_train) # Mean squared error #loss = torch.mean((y_hat-labels)**2)  sqrt to change it to rmse
                loss_train.backward()
                optimizer.step()
                optimizer.zero_grad()
                step= step+ 1
                loss_list.append(loss_train.item())
                if step % 100 == 0 :
                    print(sum(loss_list)/len(loss_list))
    #         print("Epoch {} train MSE: {:.7f}".format(epoch, sum(loss_list)/len(loss_list)))

            
    return model

In [8]:
def get_state_result(state, model):
    pred_test = []
    samples = len(tuple(dataset))
    forecast_matrix = pd.DataFrame(columns = ['date'])
    loss_fn = torch.nn.MSELoss()
    
    for i in tqdm(range(math.ceil(test_ratio*samples)-(batch_size))):
        #print(results)
        total_loss = []
        results = pd.DataFrame(columns = ['date'])
        window = i +1

        train_loader_d, test_loader_d, static_edge_index_d,edge_weights_d,test_dates = temporal_signal_split_def(dataset,dates, i, train_ratio=trainratio,
                                                                                                                 batch_size=batch_size)
        results['date'] = pd.to_datetime(test_dates[0][0], format='%Y%m%d')

        model.eval()
        for encoder_inputs, labels in test_loader_d:
            # Get model predictions
            y_hat = model(encoder_inputs, static_edge_index_d,edge_weights_d)
            
            pred_test.append(y_hat[0,state,:])
            
            # Mean squared error
            loss = loss_fn(y_hat, labels)
            total_loss.append(loss.item())

        results['window'+str(i)] = pred_test[i].cpu().detach().numpy()

        if(i==0):
            forecast_matrix = pd.merge(forecast_matrix, results, on=['date'], how='right')

        else:
            forecast_matrix = pd.merge(forecast_matrix, results, on=['date'], how='outer')
    
    return forecast_matrix
            

In [9]:
# forecast_matrix.set_index('date', inplace=True)
def predictions(forecast_matrix):
    forecast_matrix.set_index('date', inplace=True)
    start = 0
    end = 4
    daily_pred = []
    window = 0

    for i in range(len(forecast_matrix.columns)-4):
        forecast_matrix_temp = forecast_matrix.iloc[start:end]

        date = forecast_matrix_temp.tail(1).index.item()
        l_row = forecast_matrix_temp.tail(1)


        daily_pred.append([date,l_row.iloc[0][window],l_row.iloc[0][window+1],l_row.iloc[0][window+2],l_row.iloc[0][window+3]])
        window = window + 1
        start = end 
        end = end+1    

    daily_predictions = pd.DataFrame(daily_pred, columns=[ 'Date',str('week4_GNN'),str('week3_GNN'),
                                                                  str('week2_GNN'),
                                                                  str('week1_GNN')])
    daily_predictions['Date'] = pd.to_datetime(daily_predictions['Date'])
    daily_nn = daily_predictions
    return daily_nn


In [10]:
def get_observed(state,daily_nn):
   
    data = pd.read_csv("state_weekly_data.csv")
#     data = data.loc[data['Province_State'] == "Georgia"]
    data = data[["Date","Province_State", "Deaths","Confirmed"]]
    indexAge = data[(data['Province_State'] == 'Recovered') | (data['Province_State'] == 'Alaska') | (data['Province_State'] == 'Hawaii') | (data['Province_State'] == 'Puerto Rico') | (data['Province_State'] == 'Virgin Islands')
                    | (data['Province_State'] == 'American Samoa') | (data['Province_State'] == 'Diamond Princess')| (data['Province_State'] == 'Grand Princess')| (data['Province_State'] == 'Guam')| (data['Province_State'] == 'Northern Mariana Islands')].index
    data.drop(indexAge , inplace=True)
    data.reset_index(inplace=True)
    data = data.drop(columns = ['index'])
    data['Date'] = pd.to_datetime(data['Date'])
    
    state_to_number ={'Alabama':0, 'Arizona':1, 'Arkansas':2,
       'California':3, 'Colorado':4, 'Connecticut':5, 'Delaware':6, 'District of Columbia':7, 'Florida':8, 
        'Georgia':9, 'Idaho':10, 'Illinois':11, 'Indiana':12, 'Iowa':13, 'Kansas':14, 'Kentucky':15, 'Louisiana':16,
        'Maine':17, 'Maryland':18,'Massachusetts':19, 'Michigan':20, 'Minnesota':21, 'Mississippi':22,
       'Missouri':23, 'Montana':24, 'Nebraska':25, 'Nevada':26, 'New Hampshire':27,
       'New Jersey':28, 'New Mexico':29, 'New York':30, 'North Carolina':31,
       'North Dakota':32,'Ohio':33, 'Oklahoma':34,'Oregon':35, 'Pennsylvania':36,
       'Rhode Island':37, 'South Carolina':38, 'South Dakota':39, 'Tennessee':40,
       'Texas':41, 'Utah':42, 'Vermont':43, 'Virginia':44,
       'Washington':45, 'West Virginia':46, 'Wisconsin':47, 'Wyoming':48}

    # Use the map() method to apply the mapping to the state column
    data['State'] = data['Province_State'].map(state_to_number)
    data = data.drop(['Province_State'], axis=1)
    data = data.loc[data['State'] == state]
  
    observed = data[['Date','Deaths']]
    data['Deaths'] = data['Deaths'].abs()
    observed['Deaths'] = data['Deaths'].abs()
    data.reset_index(inplace=True, drop= True)
    
    observed.rename(columns={'Deaths': 'observed'}, inplace=True)
    observed.reset_index(inplace=True, drop= True)
   
    daily_nn[['week4_GNN','week3_GNN','week2_GNN','week1_GNN']] = daily_nn[['week4_GNN','week3_GNN','week2_GNN','week1_GNN']].apply(lambda x: (x*death_stds)+death_mean)
    observed['Date'] = pd.to_datetime(observed['Date'])
    matrix = reduce(lambda x,y: pd.merge(x,y, on='Date'), [observed, daily_nn])
    matrix = matrix[matrix['observed'] != 0]
    matrix.reset_index(inplace=True, drop= True)
    return matrix

In [11]:
def df_style(val):
    return "font-weight: bold"

def smape(actual,forecast): 
    return 100/len(actual) * np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast)))

def mape(y_true,y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mae(y_true,y_pred):
    error = 0
    for i in range(len(y_true)):
        error += abs(y_true[i] - y_pred[i])
    return (float((error / len(y_true))))

def rmse(y_true,y_pred):
    mse = sklearn.metrics.mean_squared_error(y_true, y_pred)   
    rmse = math.sqrt(mse) 
    return rmse

In [12]:
def rmse_results(matrix):
    results_rmse = {}
    for i in ('GNN','GNN'):
        results_rmse[i] = []
        for j in('week1','week2','week3','week4'):
            results_rmse[i].append(rmse(matrix['observed'],matrix[j+'_'+i]))
    results_rmse = pd.DataFrame(results_rmse)
    results_rmse.index += 1 
    results_rmse.loc['Average_RMSE'] = results_rmse.mean()
    last_row = pd.IndexSlice[results_rmse.index[results_rmse.index == "Average_RMSE"], :]
    results_rmse.style.applymap(df_style, subset=last_row)
    
    return results_rmse

In [13]:
#---------MAE------------#
def mae_results(matrix):
    results_mae = {}
    for i in ('GNN','GNN'):
        results_mae[i] = []
        for j in('week1','week2','week3','week4'):
            results_mae[i].append(mae(matrix['observed'],matrix[j+'_'+i]))
    results_mae = pd.DataFrame(results_mae)
    results_mae.index += 1 
    results_mae.loc['Average_MAE'] = results_mae.mean()
    last_row = pd.IndexSlice[results_mae.index[results_mae.index == "Average_MAE"], :]
    results_mae.style.applymap(df_style, subset=last_row)
    
    return results_mae


In [14]:
#---------SMAPE------------#
def smape_results(matrix):
    results_smape = {}
    for i in ('GNN','GNN'):
        results_smape[i] = []
        for j in('week1','week2','week3','week4'):
            results_smape[i].append(smape(matrix['observed'],matrix[j+'_'+i]))
    results_smape = pd.DataFrame(results_smape)
    results_smape.index += 1 
    results_smape.loc['Average_sMAPE'] = results_smape.mean()
    last_row = pd.IndexSlice[results_smape.index[results_smape.index == "Average_sMAPE"], :]
    results_smape.style.applymap(df_style, subset=last_row)
    
    return results_smape


In [ ]:
import warnings
warnings.filterwarnings("ignore")
fix_seed = 1000

random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

batch_sizes = [2,4,8,16]
hidden_unitss = [32,16,8,4,2]
trainratio = 0.8
test_ratio = 0.2
node_f = 2
periods = 4
epochs = 100
past_values = 6
dataset,dates,death_stds,death_mean = data_loader(past_values,periods)
for batch_size in batch_sizes:
    for hidden_units in hidden_unitss:
        model = train(node_f,periods,batch_size,past_values,hidden_units,trainratio,test_ratio,epochs,dataset,dates)
        for state in range(49):
            forecast_matrix = get_state_result(state, model)
            daily_nn = predictions(forecast_matrix)
            matrix = get_observed(state,daily_nn)
            smape_results_state = smape_results(matrix)
            mae_results_state = mae_results(matrix)
            rmse_results_state = rmse_results(matrix)
            print("STATE: ",state)
            print(past_values,batch_size,hidden_units)
            print(smape_results_state)
            print(mae_results_state)
            print(rmse_results_state)
            print("________________________________________")